In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, results as rs

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=3131792132272) was called 2 times


2021-01-08 18:52:49,363 - NumExpr defaulting to 8 threads.


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=3131792133136) was called 1 times


In [7]:
fa,unmapped, end_time = feat.preprocess_faults(scs_raw)

#fa= fa[fa['timestamp']<(fa['timestamp'].max()-pd.to_timedelta(7, unit='D'))]

Running: Function "preprocess_faults" (id=3131792133280) was called 1 times
Running: Function "load_tote_lookup" (id=3131792133856) was called 1 times
Running: Function "load_module_lookup" (id=3131789298704) was called 1 times
Running: Function "load_ID_lookup" (id=3131792130544) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

In [ ]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights = [0.7,0.3],duration_thres=0)

df = feat.log_totes(df) 

In [ ]:
df_week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]

In [ ]:
X,y = md.gen_feat_var(df_week,target = 'Availability', features = ['Faults','Totes'])


In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 10)

In [ ]:
R2_OOS

In [ ]:
Output = rs.create_output(fa_PTT,Coeff,end_time)

# Plotting

In [ ]:
Output['Total'] = Output[:][Output.columns[8:]].sum(axis=1)

In [ ]:
Total = Output[['Asset Code','Total']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(y='Total',x = 'Asset Code',data = Total.sort_values(by='Total',ascending=False).head(10),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
PTT081 = Output[['Asset Code','PTT081']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(y='PTT081',x = 'Asset Code',data = PTT081.sort_values(by='PTT081',ascending=False).head(5),color = 'r')
plt.ylabel('Lost Singles')
plt.title('PTT081 Lost Singles Last 12 Hours')

In [ ]:
Total_colour = Output[['Tote Colour','Total']].groupby('Tote Colour').sum().reset_index()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(y='Total',x = 'Tote Colour',data = Total_colour.sort_values(by='Total',ascending=False),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
Total_colour

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
sns.lmplot(x='TOTES', y='Availability', data=df_week, line_kws={"color": "red"})

In [ ]:
ax = sns.regplot(x="TOTES", y="Availability", data=df_week,
x_estimator=np.mean, logx=True, line_kws={"color": "red"})
plt.ylim(0,1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df_week, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
Shift = [0,0,0,10,10,10,20,20,20]
Weights = [[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1]]

In [ ]:
Weights[8]

In [ ]:
y

In [ ]:
at

In [ ]:
fa_floor

In [ ]:
df.drop(['Availability','Blue Tote Loss','Grey Tote Loss','timestamp'],axis=1)

In [ ]:
Outputs = {0.5:'Number One',0.4:'Number Two',-1.4:'Number 3'}

In [ ]:
 Output = Outputs[max(k for k, v in Outputs.items())]

In [ ]:
Outputs[0.456]='Numbers 4'

In [ ]:
Outputs

In [ ]:
print('Selected R2:', max(k for k, v in Outputs.items()))

In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=10)

df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights=[0.7,0.2,0.1])
df = feat.log_totes(df) 
df_2week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]
X,y = md.gen_feat_var(df_2week,target = 'Availability', features = ['Totes','Faults'])
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                    'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

Output = rs.create_output(fa_PTT,Coeff,end_time,speed = speed, picker_present = picker_present, availability = availability)

Running: Function "load_PTT_lookup" (id=3131792132560) was called 1 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3131792132992) was called 1 times
Running: Function "load_PTT_lookup" (id=3131792132560) was called 2 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3131792132992) was called 2 times
Running: Function "load_PTT_lookup" (id=3131792132560) was called 3 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3131792132992) was called 3 times
Running: Function "load_PTT_lookup" (id=3131792132560) was called 4 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3131792132992) was called 4 times
Running: Function "load_PTT_lookup" (id=3131792132560) was called 5 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3131792132992) was called 5 times
Running: Function "load_PTT_lookup" (id=3131792132560) was called 6 times
Previous Hours Weighted
Running

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)



Cross Validation Scores: 
 
        R2 Scores
1       0.527908
2       0.499346
3       0.338645
4       0.532183
5       0.517132
Mean    0.483043
STD     0.073084
                                 OLS Regression Results                                
Dep. Variable:           Availability   R-squared (uncentered):                   0.965
Model:                            OLS   Adj. R-squared (uncentered):              0.963
Method:                 Least Squares   F-statistic:                              690.8
Date:                Fri, 08 Jan 2021   Prob (F-statistic):                        0.00
Time:                        18:55:47   Log-Likelihood:                          5399.3
No. Observations:                8152   AIC:                                 -1.018e+04
Df Residuals:                    7843   BIC:                                     -8016.
Df Model:                         309                                                  
Covariance Type:            nonrobust     

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


NameError: name 'speed' is not defined

In [9]:
R2_OOS

0.419371400950143